In [1]:
from google.colab import drive

try:
    MOUNTED
except NameError as e:
    drive.mount('/gdrive')
    MOUNTED = True

Mounted at /gdrive


In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

path='../gdrive/My Drive/데이터마이닝/'

In [3]:
df_dataset = pd.read_csv(path + "추천영화데이터_단순+가중평균.csv")
data = pd.read_csv(path + "추천영화데이터_단순+가중평균.csv")

In [4]:
df_da = pd.read_csv(path + "추천영화데이터_단순+가중평균.csv")
idx_nm1 = df_da[df_da['영화제목'] == '명당'].index
df_da1 = df_da.drop(idx_nm1)
idx_nm1 = df_da1[df_da1['영화제목'] == '천문: 하늘에 묻는다'].index
df_da = df_da1.drop(idx_nm1)
df_da.reset_index(inplace=True)
df_da = df_da.drop(['index'],axis=1)

In [5]:
df_da

,영화제목,장르,감독,상영시간,영화평점,감성점수평균,감성점수가중평균
0,아저씨,액션 범죄 드라마,이정범,119분,9.24,78.674882,71.683266
1,실버라이닝 플레이북,코미디 드라마 멜로로맨스,데이비드 O. 러셀,122분,8.60,88.825750,83.023837
2,콜 미 바이 유어 네임,드라마 멜로로맨스,루카 구아다니노,132분,7.89,81.817257,82.178666
3,악인전,범죄 액션,이원태,110분,7.88,49.127169,49.240255
4,박화영,드라마,이환,99분,7.72,29.963238,28.578693
...,...,...,...,...,...,...,...
675,유스,드라마,파올로 소렌티노,124분,8.34,79.456340,81.367276
676,워크래프트: 전쟁의 서막,액션 모험 판타지,던칸 존스,122분,8.66,68.697979,64.788079
677,만신,다큐멘터리 드라마,박찬경,104분,8.04,70.912129,57.262420
678,내부자들,범죄 드라마,우민호,130분,9.00,51.928082,50.821093


In [6]:
df_dataset

,영화제목,장르,감독,상영시간,영화평점,감성점수평균,감성점수가중평균
0,아저씨,액션 범죄 드라마,이정범,119분,9.24,78.674882,71.683266
1,실버라이닝 플레이북,코미디 드라마 멜로로맨스,데이비드 O. 러셀,122분,8.60,88.825750,83.023837
2,콜 미 바이 유어 네임,드라마 멜로로맨스,루카 구아다니노,132분,7.89,81.817257,82.178666
3,악인전,범죄 액션,이원태,110분,7.88,49.127169,49.240255
4,박화영,드라마,이환,99분,7.72,29.963238,28.578693
...,...,...,...,...,...,...,...
677,유스,드라마,파올로 소렌티노,124분,8.34,79.456340,81.367276
678,워크래프트: 전쟁의 서막,액션 모험 판타지,던칸 존스,122분,8.66,68.697979,64.788079
679,만신,다큐멘터리 드라마,박찬경,104분,8.04,70.912129,57.262420
680,내부자들,범죄 드라마,우민호,130분,9.00,51.928082,50.821093


In [7]:
list_genre = df_da['장르'].str.split(' ',expand = True)
list_genre

,0,1,2,3,4,5,6
0,액션,범죄,드라마,None,None,None,None
1,코미디,드라마,멜로로맨스,None,None,None,None
2,드라마,멜로로맨스,None,None,None,None,None
3,범죄,액션,None,None,None,None,None
4,드라마,None,None,None,None,None,None
...,...,...,...,...,...,...,...
675,드라마,None,None,None,None,None,None
676,액션,모험,판타지,None,None,None,None
677,다큐멘터리,드라마,None,None,None,None,None
678,범죄,드라마,None,None,None,None,None


In [8]:
list_genre = list_genre.applymap(str)
genres = []
for i in range(0,7):
  list_genre[i]
  for j in range(0,680):
    if (list_genre[i][j] not in genres) and list_genre[i][j] != 'None' and list_genre[i][j] != 'nan' and list_genre[i][j] != ' ':
      genres.append(list_genre[i][j])
    else:
      pass
genres = list(filter(None, genres))
len(genres)

18

In [9]:
seperated_genres = df_da['장르'].str.get_dummies(' ')
dataset = pd.concat([df_da, seperated_genres], axis = 1, sort = False)
dataset

,영화제목,장르,감독,상영시간,영화평점,감성점수평균,감성점수가중평균,가족,공포,느와르,다큐멘터리,드라마,멜로로맨스,모험,뮤지컬,미스터리,범죄,서사,서스펜스,스릴러,애니메이션,액션,전쟁,코미디,판타지
0,아저씨,액션 범죄 드라마,이정범,119분,9.24,78.674882,71.683266,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1,실버라이닝 플레이북,코미디 드라마 멜로로맨스,데이비드 O. 러셀,122분,8.60,88.825750,83.023837,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0
2,콜 미 바이 유어 네임,드라마 멜로로맨스,루카 구아다니노,132분,7.89,81.817257,82.178666,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,악인전,범죄 액션,이원태,110분,7.88,49.127169,49.240255,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,박화영,드라마,이환,99분,7.72,29.963238,28.578693,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,유스,드라마,파올로 소렌티노,124분,8.34,79.456340,81.367276,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
676,워크래프트: 전쟁의 서막,액션 모험 판타지,던칸 존스,122분,8.66,68.697979,64.788079,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
677,만신,다큐멘터리 드라마,박찬경,104분,8.04,70.912129,57.262420,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
678,내부자들,범죄 드라마,우민호,130분,9.00,51.928082,50.821093,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   영화제목      680 non-null    object 
 1   장르        678 non-null    object 
 2   감독        680 non-null    object 
 3   상영시간      680 non-null    object 
 4   영화평점      680 non-null    float64
 5   감성점수평균    680 non-null    float64
 6   감성점수가중평균  680 non-null    float64
 7   가족        680 non-null    int64  
 8   공포        680 non-null    int64  
 9   느와르       680 non-null    int64  
 10  다큐멘터리     680 non-null    int64  
 11  드라마       680 non-null    int64  
 12  멜로로맨스     680 non-null    int64  
 13  모험        680 non-null    int64  
 14  뮤지컬       680 non-null    int64  
 15  미스터리      680 non-null    int64  
 16  범죄        680 non-null    int64  
 17  서사        680 non-null    int64  
 18  서스펜스      680 non-null    int64  
 19  스릴러       680 non-null    int64  
 20  애니메이션     680 non-null    int64 

In [11]:
min_max_scaler = MinMaxScaler()
dataset[['감성점수평균']] = min_max_scaler.fit_transform(dataset[['감성점수평균']])
dataset[['영화평점']] = min_max_scaler.fit_transform(dataset[['영화평점']])

In [12]:
dataset

,영화제목,장르,감독,상영시간,영화평점,감성점수평균,감성점수가중평균,가족,공포,느와르,다큐멘터리,드라마,멜로로맨스,모험,뮤지컬,미스터리,범죄,서사,서스펜스,스릴러,애니메이션,액션,전쟁,코미디,판타지
0,아저씨,액션 범죄 드라마,이정범,119분,0.943730,0.784352,71.683266,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1,실버라이닝 플레이북,코미디 드라마 멜로로맨스,데이비드 O. 러셀,122분,0.840836,0.912328,83.023837,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0
2,콜 미 바이 유어 네임,드라마 멜로로맨스,루카 구아다니노,132분,0.726688,0.823969,82.178666,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,악인전,범죄 액션,이원태,110분,0.725080,0.411831,49.240255,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,박화영,드라마,이환,99분,0.699357,0.170223,28.578693,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,유스,드라마,파올로 소렌티노,124분,0.799035,0.794204,81.367276,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
676,워크래프트: 전쟁의 서막,액션 모험 판타지,던칸 존스,122분,0.850482,0.658569,64.788079,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
677,만신,다큐멘터리 드라마,박찬경,104분,0.750804,0.686483,57.262420,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
678,내부자들,범죄 드라마,우민호,130분,0.905145,0.447143,50.821093,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [13]:
dataset[['감성점수평균', '가족', '공포', '느와르', '다큐멘터리', '드라마', '멜로로맨스', '모험', '뮤지컬', '미스터리', '범죄', '서사', '서스펜스', '스릴러', '애니메이션', '액션', '전쟁', '코미디', '판타지']]

,감성점수평균,가족,공포,느와르,다큐멘터리,드라마,멜로로맨스,모험,뮤지컬,미스터리,범죄,서사,서스펜스,스릴러,애니메이션,액션,전쟁,코미디,판타지
0,0.784352,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1,0.912328,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0
2,0.823969,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0.411831,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,0.170223,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,0.794204,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
676,0.658569,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
677,0.686483,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
678,0.447143,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [14]:
# features = dataset[['영화평점','감성점수평균','가족', '공포', '느와르', '다큐멘터리', '드라마', '멜로로맨스', '모험', '뮤지컬', '미스터리', '범죄', '서사', '서스펜스', '스릴러', '애니메이션', '액션', '전쟁', '코미디', '판타지']].astype(float)
features = dataset[['영화평점','감성점수평균']]
# Scaling the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features)

# # Using TSNE
tsne = TSNE(n_components=2)
transformed_genre = tsne.fit_transform(scaled_data)
# transformed_genre = features

In [15]:
scaled_data[0]

array([1.03774306, 0.26721749])

In [ ]:
#KMeans - Elbow Method 영화평점 + 감성점수평균 /min-max
distortions = []
K = range(1,100)
for k in K:
    kmean = KMeans(n_clusters=k)
    kmean.fit(features)
    distortions.append(kmean.inertia_)
fig = px.line(x=K,y=distortions,title='The Elbow Method Showing The Optimal K',
              labels={'x':'No of Clusters','y':'Distortions'})
fig.show()

In [ ]:
#KMeans - Elbow Method
distortions = []
K = range(1,100)
for k in K:
    kmean = KMeans(n_clusters=k)
    kmean.fit(scaled_data)
    distortions.append(kmean.inertia_)
fig = px.line(x=K,y=distortions,title='The Elbow Method Showing The Optimal K',
              labels={'x':'No of Clusters','y':'Distortions'})
fig.show()

In [ ]:
#장르만 이용
#Kmeans
cluster = KMeans(n_clusters=15)
group_pred = cluster.fit_predict(transformed_genre)

tsne_df = pd.DataFrame(np.column_stack((transformed_genre,group_pred,dataset['영화제목'],dataset['장르'],dataset['감성점수평균'])),columns=['X','Y','Group','Title','Genres','avg'])

fig = px.scatter(tsne_df,x='X',y='Y',hover_data=['Title','Genres','avg'],color='Group',
                 color_discrete_sequence=px.colors.cyclical.Phase)
fig.show()

In [ ]:
#감성점수평점 + 장르
#Kmeans
cluster = KMeans(n_clusters=20)
group_pred = cluster.fit_predict(scaled_data)

tsne_df = pd.DataFrame(np.column_stack((transformed_genre,group_pred,dataset['영화제목'],dataset['장르'],dataset['감성점수평균'])),columns=['X','Y','Group','Title','Genres','avg'])

fig = px.scatter(tsne_df,x='X',y='Y',hover_data=['Title','Genres','avg'],color='Group',
                 color_discrete_sequence=px.colors.cyclical.IceFire)
fig.show()

In [ ]:
#영화평점 + 감성점수평균 + 장르
#Kmeans
cluster = KMeans(n_clusters=14)
group_pred = cluster.fit_predict(scaled_data)

tsne_df = pd.DataFrame(np.column_stack((transformed_genre,group_pred,dataset['영화제목'],dataset['장르'],dataset['감성점수평균'])),columns=['X','Y','Group','Title','Genres','avg'])

fig = px.scatter(tsne_df,x='X',y='Y',hover_data=['Title','Genres','avg'],color='Group',
                 color_discrete_sequence=px.colors.cyclical.IceFire)
fig.show()

In [ ]:
#영화평점 + 감성점수평균
#Kmeans
cluster = KMeans(n_clusters=6)
group_pred = cluster.fit_predict(features)

tsne_df = pd.DataFrame(np.column_stack((transformed_genre,group_pred,dataset['영화제목'],dataset['장르'],dataset['감성점수평균'])),columns=['X','Y','Group','Title','Genres','avg'])

fig = px.scatter(tsne_df,x='X',y='Y',hover_data=['Title','Genres','avg'],color='Group',
                 color_discrete_sequence=px.colors.cyclical.HSV_r)
fig.show()

In [22]:
#영화평점 + 감성점수평균 min-max
#Kmeans
cluster = KMeans(n_clusters=6)
group_pred = cluster.fit_predict(transformed_genre)

tsne_df = pd.DataFrame(np.column_stack((transformed_genre,group_pred,dataset['영화제목'],dataset['장르'],dataset['감성점수평균'])),columns=['X','Y','Group','Title','Genres','avg'])

fig = px.scatter(tsne_df,x='X',y='Y',hover_data=['Title','Genres','avg'],color='Group',
                 color_discrete_sequence=px.colors.cyclical.HSV_r)
fig.show()

In [17]:
from math import sqrt

# calculate the Euclidean distance between two vectors
# row = [x, y, type]
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)
 
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

In [18]:
row1 = np.array(transformed_genre)[50]
print(range(len(row1)-1))

range(0, 1)


In [20]:
row0 = np.array(transformed_genre)[50]
neighbors = get_neighbors(np.array(transformed_genre), row0, 11)
for neighbor in neighbors:
	print(neighbor)

[-10.196668   -0.2500782]
[-9.850817   1.1151289]
[-8.556219    0.88023734]
[-10.698207    1.8407677]
[-12.791258    0.5703789]
[-11.66691    -2.5933554]
[-10.161255    2.8993394]
[-12.775211   -2.2466886]
[-12.924731    1.6751043]
[-11.743041    2.7930334]
[-11.124994    3.3054223]


In [ ]:
row0 = np.array(transformed_genre)[50]
neighbors = get_neighbors(np.array(transformed_genre), row0, 11)
for neighbor in neighbors:
	print(neighbor)

[0.87299035 0.76153154]
[0.86977492 0.7523484 ]
[0.86012862 0.75479294]
[0.87459807 0.74532921]
[0.86495177 0.77837713]
[0.88585209 0.77707392]
[0.89228296 0.7541644 ]
[0.86977492 0.78231894]
[0.87459807 0.73837467]
[0.89228296 0.77489201]
[0.89228296 0.74701946]


In [ ]:
print(dataset.loc[50])

영화제목            어벤져스
장르            액션  모험
감독             조스 웨던
상영시간           142분 
영화평점         0.87299
감성점수평균      0.761532
감성점수가중평균     79.3129
가족                 0
공포                 0
느와르                0
다큐멘터리              0
드라마                0
멜로로맨스              0
모험                 1
뮤지컬                0
미스터리               0
범죄                 0
서사                 0
서스펜스               0
스릴러                0
애니메이션              0
액션                 1
전쟁                 0
코미디                0
판타지                0
Name: 50, dtype: object


In [ ]:
for i in range(len(transformed_genre)):
  for j in range(11):
    if (np.array_equal(np.array(transformed_genre)[i],neighbors[j])):
      if not (np.array_equal(dataset.loc[i],dataset.loc[50])):
        print(dataset[['영화제목','장르','감독','상영시간','영화평점','감성점수평균']].loc[i])

영화제목              너의 이름은.
장르        애니메이션 드라마 멜로로맨스
감독                신카이 마코토
상영시간                106분 
영화평점             0.869775
감성점수평균           0.752348
Name: 17, dtype: object
영화제목             아이언맨
장르         액션 드라마 판타지
감독              존 파브로
상영시간            125분 
영화평점         0.892283
감성점수평균       0.747019
Name: 97, dtype: object
영화제목      분노의 질주: 더 맥시멈
장르             액션 모험 범죄
감독                저스틴 린
상영시간              130분 
영화평점           0.874598
감성점수평균         0.738375
Name: 313, dtype: object
영화제목      그것만이 내 세상
장르          코미디 드라마
감독              최성현
상영시간          120분 
영화평점       0.892283
감성점수평균     0.754164
Name: 335, dtype: object
영화제목                  드래곤 길들이기 2
장르        애니메이션 액션 모험 코미디 가족 판타지
감독                       딘 데블로이스
상영시간                       101분 
영화평점                    0.892283
감성점수평균                  0.774892
Name: 389, dtype: object
영화제목            마당을 나온 암탉
장르        애니메이션 가족 모험 드라마
감독                    오성윤
상영시간                 93분 
영화평점             0.8858

In [ ]:
for i in range(len(transformed_genre)):
  for j in range(11):
    if (np.array_equal(np.array(transformed_genre)[i],neighbors[j])):
      if not (np.array_equal(dataset.loc[i],dataset.loc[50])):
        print(dataset[['영화제목','장르','감독','상영시간','영화평점','감성점수평균']].loc[i])

영화제목        아이언맨 3
장르           모험 액션
감독           셰인 블랙
상영시간         129분 
영화평점      0.882637
감성점수평균    0.730101
Name: 112, dtype: object
영화제목       퍼스트 어벤져
장르           액션 모험
감독           조 존스톤
상영시간         123분 
영화평점      0.710611
감성점수평균     0.87165
Name: 123, dtype: object
영화제목      트랜스포머: 패자의 역습
장르                액션 모험
감독               마이클 베이
상영시간              149분 
영화평점           0.762058
감성점수평균         0.703102
Name: 160, dtype: object
영화제목      캡틴 아메리카: 윈터 솔져
장르                액션 모험 
감독                  조 루소
상영시간               136분 
영화평점            0.893891
감성점수평균          0.728836
Name: 183, dtype: object
영화제목      해적: 바다로 간 산적
장르               모험 액션
감독                 이석훈
상영시간             130분 
영화평점          0.789389
감성점수평균        0.773093
Name: 185, dtype: object
영화제목       스타트렉 다크니스
장르            액션 모험 
감독        J.J. 에이브럼스
상영시간           132분 
영화평점        0.900322
감성점수평균      0.648747
Name: 204, dtype: object
영화제목          아쿠아맨
장르          액션 모험 
감독           제임스 완
상영시간   

In [21]:
for i in range(len(transformed_genre)):
  for j in range(11):
    if (np.array_equal(np.array(transformed_genre)[i],neighbors[j])):
      if not (np.array_equal(dataset.loc[i],dataset.loc[50])):
        print(dataset[['영화제목','장르','감독','상영시간','영화평점','감성점수평균']].loc[i])

영화제목              너의 이름은.
장르        애니메이션 드라마 멜로로맨스
감독                신카이 마코토
상영시간                106분 
영화평점             0.869775
감성점수평균           0.752348
Name: 17, dtype: object
영화제목             아이언맨
장르         액션 드라마 판타지
감독              존 파브로
상영시간            125분 
영화평점         0.892283
감성점수평균       0.747019
Name: 97, dtype: object
영화제목       라이프 오브 파이
장르        모험 드라마 판타지
감독                이안
상영시간           127분 
영화평점        0.882637
감성점수평균      0.735387
Name: 273, dtype: object
영화제목      분노의 질주: 더 맥시멈
장르             액션 모험 범죄
감독                저스틴 린
상영시간              130분 
영화평점           0.874598
감성점수평균         0.738375
Name: 313, dtype: object
영화제목      그것만이 내 세상
장르          코미디 드라마
감독              최성현
상영시간          120분 
영화평점       0.892283
감성점수평균     0.754164
Name: 335, dtype: object
영화제목                  드래곤 길들이기 2
장르        애니메이션 액션 모험 코미디 가족 판타지
감독                       딘 데블로이스
상영시간                       101분 
영화평점                    0.892283
감성점수평균                  0.774892
N

In [ ]:
for i in range(len(transformed_genre)):
  if (np.array_equal(transformed_genre[i],neighbors[0]) or np.array_equal(transformed_genre[i],neighbors[1]) or np.array_equal(transformed_genre[i],neighbors[2])):
    if not (np.array_equal(dataset.loc[i],dataset.loc[0])):
      print(dataset.loc[i])

영화제목             내 사랑
장르          드라마 멜로로맨스
감독            에이슬링 월쉬
상영시간            115분 
영화평점         0.932476
감성점수평균       0.800237
감성점수가중평균      78.8067
가족                  0
공포                  0
느와르                 0
다큐멘터리               0
드라마                 1
멜로로맨스               1
모험                  0
뮤지컬                 0
미스터리                0
범죄                  0
서사                  0
서스펜스                0
스릴러                 0
애니메이션               0
액션                  0
전쟁                  0
코미디                 0
판타지                 0
Name: 36, dtype: object
영화제목            어벤져스
장르            액션  모험
감독             조스 웨던
상영시간           142분 
영화평점         0.87299
감성점수평균      0.761532
감성점수가중평균     79.3129
가족                 0
공포                 0
느와르                0
다큐멘터리              0
드라마                0
멜로로맨스              0
모험                 1
뮤지컬                0
미스터리               0
범죄                 0
서사                 0
서스펜스               0
스릴러                0
애니메이션 

In [ ]:
dataset.loc[0]

영화제목              아저씨
장르          액션 범죄 드라마
감독                이정범
상영시간            119분 
영화평점          0.94373
감성점수평균       0.784352
감성점수가중평균      71.6833
가족                  0
공포                  0
느와르                 0
다큐멘터리               0
드라마                 1
멜로로맨스               0
모험                  0
뮤지컬                 0
미스터리                0
범죄                  1
서사                  0
서스펜스                0
스릴러                 0
애니메이션               0
액션                  1
전쟁                  0
코미디                 0
판타지                 0
Name: 0, dtype: object

In [ ]:
np.array_equal(dataset.loc[0],dataset.loc[0])

True

In [ ]:
for i in range(len(transformed_genre)):
  if dataset.loc[i]['영화제목'] == '위플래쉬':
    print(i)

244


In [ ]:
np.array_equal(dataset.loc[244],dataset.loc[0])

False

In [ ]:
group_pred

array([ 3,  1,  1,  3,  4,  4,  1,  4,  1,  2,  4,  3,  0,  3,  1,  4,  3,
        1,  6,  4,  0,  3,  1,  4,  2,  1,  4,  4,  4,  0,  1,  4,  0,  1,
        4,  4,  1,  1, 10,  0,  1,  0,  1,  1,  0,  2,  0,  2,  1,  1,  0,
        4,  4,  0,  2,  5,  1,  5,  0,  4,  5,  0,  2,  0, 11,  3,  0,  0,
        4,  7,  2,  4,  0,  5,  2,  8,  0, 10,  4,  1,  1,  2,  0,  5,  4,
        4,  1,  9,  2,  4,  2,  4,  3,  2,  5,  1,  2,  0,  4,  7,  7,  4,
        2,  1,  4,  0,  1,  1,  6,  0,  1,  1,  0,  1,  4,  2,  4,  5,  0,
        4,  4,  4,  7,  0,  4,  4,  0,  0,  1,  0,  1,  5,  4,  4,  4,  2,
        5,  1, 11,  3,  6,  4,  4,  2,  4,  1,  0,  2,  0,  2,  6,  1,  4,
        4,  2,  4,  0,  4,  7,  0,  0,  4,  4,  1, 11,  2,  4,  0,  6,  1,
        4,  3, 10,  1,  2,  4,  0,  4,  4,  0,  7,  2,  0,  0,  4,  0,  4,
        3,  0,  6,  3,  3,  4,  4,  4,  4,  3,  4,  0,  4,  4,  4,  2,  0,
        0,  5,  1,  5,  5,  0,  2,  0,  1,  1,  4,  1,  2,  2, 10,  1,  2,
        4,  2,  2,  1,  1